In [0]:
%pip install sqlalchemy
%pip install plotnine
# %pip install mlflow


Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Obtaining dependency information for sqlalchemy from https://files.pythonhosted.org/packages/4d/21/87bcad723070f7cd5f9d45fb05557596aa1d23d19eef078b13edc9e31813/SQLAlchemy-2.0.30-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for greenlet!=0.4.17 from https://files.pythonhosted.org/packages/c6/1f/12d5a6cc26e8b483c2e7975f9c22e088ac735c0d8dcb8a8f72d31a4e5f04/greenlet-3.0.3-cp311-cp311-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/3.2 MB ? eta -:--:--
   ━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/3.2 MB 4.1 MB/s eta 0:00:01
   ━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.5/3.2 MB 7.2 MB/s eta 0:00:01
   ━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.9/3.2 MB 8.7 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━ 1.4/3.2 MB 9.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━

In [0]:
dbutils.library.restartPython()

In [0]:
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine
import datetime as dt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from plotnine import ggplot, geom_point, aes, geom_line, xlab
from pyspark.sql.functions import when, col
from pyspark.sql import functions as F

In [0]:
import pandas as pd
import numpy as np

def get_data(con,query):

  cur=con.cursor()
  return pd.DataFrame(cur.execute(query).fetch_dataframe())

def get_mode_per_column(dataframe, group_cols, col):
    return (dataframe.fillna(-1) # NaN placeholder to keep group 
            .groupby(group_cols + [col])
            .size()
            .to_frame('count')
            .reset_index()
            .sort_values('count', ascending=False)
            .drop_duplicates(subset=group_cols)
            .drop(columns=['count'])
            .sort_values(group_cols)
            .replace(-1, np.NaN))  # restore NaNs

def max_and_join(df,vector,col_name):

    df['temp']=vector
    temp=df[['PERSON_PR_ID','temp']].groupby('PERSON_PR_ID').max()
    df=df.join(temp,on='PERSON_PR_ID',rsuffix='_').rename(columns={'temp_':col_name})
    df=df.drop(columns=['temp'])
    return df

def replace_with_unknown(df,col):
    df[col]=df[col].fillna('Unknown')
    return df

def replace_with_zero(df,col):
    df[col]=df[col].fillna(0)
    return df


In [0]:
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

In [0]:
path = "/FileStore/tables/Hilton_Segmentation/Value_full_data"
Value_df = spark.read.format("csv").option("header","true").option("inferSchema","true").load(path)

In [0]:
df_spark = pd.read_csv('RAPP_Spark_Customers.csv')

In [0]:
df_spark.replace(['(null)', 'null', 'None'], None, inplace=True)

# Convert the Pandas DataFrame to a Spark DataFrame
df_spark = spark.createDataFrame(df_spark)

# Return the converted Spark DataFrame
# df_rapp_spark.display()

In [0]:
df_spark = df_spark.withColumn('canopy_LIT_p12m', when(col('cust_stays_canopy_p12m')>=1, 1).otherwise(0))\
       .withColumn('canopy_LIT_p36m', when(col('cust_stays_canopy_p36m')>=1, 1).otherwise(0))\
       .withColumn('tempo_LIT_p12m', when(col('cust_stays_tempo_p12m')>=1, 1).otherwise(0))\
       .withColumn('tempo_LIT_p36m', when(col('cust_stays_tempo_p36m')>=1, 1).otherwise(0))\
       .withColumn('motto_LIT_p12m', when(col('cust_stays_motto_p12m')>=1, 1).otherwise(0))\
       .withColumn('motto_LIT_p36m', when(col('cust_stays_motto_p36m')>=1, 1).otherwise(0))\
       .withColumn('curio_LIT_p12m', when(col('cust_stays_curio_p12m')>=1, 1).otherwise(0))\
       .withColumn('curio_LIT_p36m', when(col('cust_stays_curio_p36m')>=1, 1).otherwise(0))\
       .withColumn('tapestry_LIT_p12m', when(col('cust_stays_tapestry_p12m')>=1, 1).otherwise(0))\
       .withColumn('tapestry_LIT_p36m', when(col('cust_stays_tapestry_p36m')>=1, 1).otherwise(0))\
       .withColumn('LIT_stays_p12m', col('cust_stays_canopy_p12m') + col('cust_stays_tempo_p12m') +
                   col('cust_stays_motto_p12m') + col('cust_stays_curio_p12m') + col('cust_stays_tapestry_p12m'))\
       .withColumn('LIT_stays_p36m', col('cust_stays_canopy_p36m') + col('cust_stays_tempo_p36m') +
                   col('cust_stays_motto_p36m') + col('cust_stays_curio_p36m') + col('cust_stays_tapestry_p36m'))\
       .withColumn('LIT_stays_p12m_flag', when(col("LIT_stays_p12m")>=1, 1).otherwise(0))\
       .withColumn('LIT_stays_p36m_flag', when(col("LIT_stays_p36m")>=1, 1).otherwise(0))\
       .withColumn('All_stays_p12m', col('CUST_DOMESTIC_STAYS_P12M') + col('CUST_INTL_STAYS_P12M'))\
       .withColumn('All_stays_p36m', col('CUST_DOMESTIC_STAYS_P36M') + col('CUST_INTL_STAYS_P36M'))\
       .withColumn('hampton_Value_p12m', when(col('cust_stays_hampton_p12m')>=1, 1).otherwise(0))\
       .withColumn('hampton_Value_p36m', when(col('cust_stays_hampton_p36m')>=1, 1).otherwise(0))\
       .withColumn('tru_Value_p12m', when(col('cust_stays_tru_p12m')>=1, 1).otherwise(0))\
       .withColumn('tru_Value_p36m', when(col('cust_stays_tru_p36m')>=1, 1).otherwise(0))\
       .withColumn('Spark_Value_p12m', when(col('cust_stays_Spark_p12m')>=1, 1).otherwise(0))\
       .withColumn('Spark_Value_p36m', when(col('cust_stays_Spark_p36m')>=1, 1).otherwise(0))\
       .withColumn('Value_stays_p12m', col('cust_stays_hampton_p12m') + col('cust_stays_tru_p12m') + col('cust_stays_Spark_p12m'))\
       .withColumn('Value_stays_p36m', col('cust_stays_hampton_p36m') + col('cust_stays_tru_p36m') + col('cust_stays_Spark_p36m'))\
       .withColumn('Value_stays_p12m_flag', when(col("Value_stays_p12m")>=1, 1).otherwise(0))\
       .withColumn('Value_stays_p36m_flag', when(col("Value_stays_p36m")>=1, 1).otherwise(0))\
       .withColumn('reward_stay_p12m', when(col('REWARD_HH_STAYS_P12M')>=1, 1).otherwise(0))\
       .withColumn('reward_stay_p36m', when(col('REWARD_HH_STAYS_P36M')>=1, 1).otherwise(0))\
       .withColumn('customer_count', when(col('MARKETABLE_CUSTOMER')>=0, 1).otherwise(0))\
       .withColumn('Cust_stays_p36m_grp', when(col("Cust_stays_p36m")>=20, 20).otherwise(col("Cust_stays_p36m")))\
       .withColumn('Cust_nights_p36m_grp', when(col("Cust_nights_p36m")>=40, 40).otherwise(col("Cust_nights_p36m")))\
       .withColumn('CUST_FOLIO_P36M_grp', F.round(col('CUST_FOLIO_P36M')/100, 0))\
       .withColumn('CUST_FOOD_P36M_grp', F.round(col('CUST_FOOD_SPEND_P36M')/100, 0))\
       .withColumn('CUST_BEVERAGE_P36M_grp', F.round(col('CUST_BEVERAGE_SPEND_P36M')/100, 0))\
       .withColumn('CUST_FOLIO_P36M_grp2', when(col('CUST_FOLIO_P36M_grp')>50, 50).otherwise(col('CUST_FOLIO_P36M_grp')))\
       .withColumn('CUST_HIGH_VALUE', when((col('CUST_FOLIO_P36M')>=3000) | (col('cust_stays_p36m')>=10) | (col('cust_nights_p36m')>=20), 1).otherwise(0))\
       .withColumn('CUST_FOOD_P36M_grp2', when(col('CUST_FOOD_P36M_grp')>100, 100).otherwise(col('CUST_FOOD_P36M_grp')))\
       .withColumn('CUST_BEVERAGE_P36M_grp2', when(col('CUST_BEVERAGE_P36M_grp')>100, 100).otherwise(col('CUST_BEVERAGE_P36M_grp')))\
       .withColumn('CUST_FOLIO_CANOPY_P36M_v2', F.nanvl(F.col("CUST_FOLIO_CANOPY_P36M"), F.lit(0)))\
       .withColumn('CUST_AVG_PTS_REDEEMED_STAY_P12M_grp', F.round(col('CUST_AVG_PTS_REDEEMED_STAY_P12M')/1000, 0))

In [0]:
df_spark = df_spark.filter(col("LIT_stays_p36m_flag") == 0)

In [0]:
Value_df = Value_df.join(df_spark, Value_df['HILTON_CUSTOMER_ID'] == df_spark['hilton_customer_id'], 'left_anti')

In [0]:
# Value_df = Value_df.limit(2000000)
Value_df = Value_df.sample(fraction = 0.15, seed=42)

In [0]:
num_rows = Value_df.count()
num_columns = len(Value_df.columns)

print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")

Number of rows: 6267796
Number of columns: 214


In [0]:
spark.conf.set("spark.sql.execution.arrow.enabled","true")

/databricks/spark/python/pyspark/sql/connect/conf.py:48: UserWarning: The SQL config 'spark.sql.execution.arrow.enabled' is NOT supported in Spark Connect


In [0]:
# # V1
clustering_columns = ['HONORS_ACTIVE_FLAG', 'HONORS_TIER', 'DISTINCT_COUNTRIES_PAST_12MONTHS', 
                      'MARKETABLE_CUSTOMER', 'COBRAND_CARDHOLDER_FLAG', 'PRIMARY_STAY_TYPE_LIFETIME', 
                      'PRIMARY_CHANNEL_LIFETIME', 'HONORS_ENROLL_YEAR', 'FIRST_CHECKOUT_YEAR', 'COBRAND_ENROLL_YEAR_POST_2018',
                      'PROMO_REGISTRANT_FLAG_P12M', 'PREFERRED_LANGUGAGE_GROUPING', 
                      'US_HONORS_INCOME_RANGE_DETAILED', 'US_HONORS_HOUSEHOLD_SIZE', 
                      'PT_REDEEMER_PAST_12MONTHS', 'CUST_FOOD_SPEND_P36M', 'CUST_BEVERAGE_SPEND_P36M',
                      'CUST_STAYS_P36M', 'CUST_NIGHTS_P36M', 'CUST_FOLIO_P36M', 'CUST_STAYS_RESORT_P36M', 
                      'CUST_STAYS_LUXURY_BRAND_DIV_P36M',  
                      'FB_FOLIO_RATIO',
                      'TOTAL_TRIP_DIST_MILES_P36M', 'EMAIL_CLICKS_P36M']

# clustering_columns = ['HONORS_ACTIVE_FLAG', 'HONORS_TIER', 'DISTINCT_COUNTRIES_PAST_12MONTHS', 
#                       'MARKETABLE_CUSTOMER', 'COBRAND_CARDHOLDER_FLAG', 'PRIMARY_STAY_TYPE_LIFETIME', 
#                       'PRIMARY_CHANNEL_LIFETIME', 'HONORS_ENROLL_YEAR', 'FIRST_CHECKOUT_YEAR', 'COBRAND_ENROLL_YEAR_POST_2018',
#                       'PROMO_REGISTRANT_FLAG_P12M', 'PREFERRED_LANGUGAGE_GROUPING',  'CUST_FOOD_SPEND_P36M', 
#                       'CUST_BEVERAGE_SPEND_P36M', 'US_HONORS_INCOME_RANGE_DETAILED', 'US_HONORS_HOUSEHOLD_SIZE', 
#                       'PT_REDEEMER_PAST_12MONTHS', 'UNIQUE_BRANDS_P12M_BUCKETS', 'FB_FOLIO_RATIO',
#                       'US_HONORS_OCCUPATION', 'US_HONORS_OCCUPATION_DETAILED', 'UNIQUE_BRANDS_P12M_BUCKETS', 'FB_FOLIO_RATIO',
#                       'UNIQUE_BRANDS_P36M_BUCKETS', 'CUST_MASS_PROMO_STAYS_P36M', 'CUST_MASS_PROMO_NIGHTS_P36M', 
#                       'CUST_STAYS_P36M', 'CUST_NIGHTS_P36M', 'CUST_FOLIO_P36M', 'CUST_STAYS_RESORT_P36M', 
#                       'CUST_STAYS_LUXURY_BRAND_DIV_P36M', 'TOTAL_TRIP_DIST_MILES_P36M', 'EMAIL_CLICKS_P36M']

profiling_columns = ['CUSTOMER_SEGMENT', 'COBRAND_SEG', 'LIFETIME_DIAMOND_FLAG', 
                     'ORGANIC_INORGANIC_FLAG', 'ENROLL_CHANNEL', 'APP_LAUNCHER_P12M', 
                     'CUSTOMER_REGION_CODE', 
                     'CUSTOMER_COUNTRY', 'HGV_OWNER_FLAG', 'PRIMARY_BRAND_LT', 
                     'US_HONORS_OCCUPATION', 'US_HONORS_OCCUPATION_DETAILED', 'US_HONORS_LIFE_STAGE',
                     'UPSCALE_PROPENSITY', 'POINTS_BALANCE_CATEGORIES', 'UNIQUE_BRANDS_P12M_BUCKETS', 
                     'UNIQUE_BRANDS_P36M_BUCKETS', 'FIRST_PREFERRED_BOOKING_RATE_P36M', 
                     'SECOND_PREFERRED_BOOKING_RATE_P36M', 'THIRD_PREFERRED_BOOKING_RATE_P31M',
                     'CUST_AVG_PTS_REDEEMED_STAY_P12M', 'CUSTOMER_VALUE_FY2022', 'CUSTOMER_VALUE_3YEARS',
                     'REWARD_HH_STAYS_P12M', 'REWARD_HH_STAYS_P36M', 'CUST_UNIQUE_BRANDS_P12M', 
                     'CUST_UNIQUE_BRANDS_P36M', 'CUST_STAYS_P12M', 'CUST_NIGHTS_P12M', 'CUST_FOLIO_P12M', 
                     'CUST_FOOD_SPEND_P12M', 'CUST_BEVERAGE_SPEND_P12M', 
                     'CUST_FOOD_SPEND_P36M', 'CUST_BEVERAGE_SPEND_P36M', 
                     'CUST_STAYS_RESORT_P12M', 'CUST_STAYS_LUXURY_BRAND_DIV_P12M', 
                     'CUST_DOMESTIC_STAYS_P12M', 'CUST_INTL_STAYS_P12M',
                     'CUST_DOMESTIC_STAYS_P36M', 'CUST_INTL_STAYS_P36M', 
                     'TOTAL_TRIP_DIST_MILES_P12M', 'TOTAL_DOMESTIC_TRIP_DIST_MILES_P12M', 
                     'TOTAL_INTL_TRIP_DIST_MILES_P12M', 'EMAIL_CLICKS_P12M', 
                     'CUST_STAYS_HAMPTON_P12M', 'CUST_STAYS_HAMPTON_P36M', 
                     'CUST_STAYS_HILTON_P12M', 'CUST_STAYS_HILTON_P36M', 'CUST_STAYS_HGI_P12M', 
                     'CUST_STAYS_HGI_P36M', 'CUST_STAYS_HOMEWOOD_P12M', 'CUST_STAYS_HOMEWOOD_P36M', 
                     'CUST_STAYS_DOUBLETREE_P12M', 'CUST_STAYS_DOUBLETREE_P36M', 'CUST_STAYS_EMBASSY_P12M', 
                     'CUST_STAYS_EMBASSY_P36M', 'CUST_STAYS_HOME2_P12M', 'CUST_STAYS_HOME2_P36M', 
                     'CUST_STAYS_TRU_P12M', 'CUST_STAYS_TRU_P36M', 'CUST_STAYS_CANOPY_P12M', 
                     'CUST_STAYS_CANOPY_P36M', 'CUST_STAYS_TAPESTRY_P12M', 'CUST_STAYS_TAPESTRY_P36M', 
                     'CUST_STAYS_CONRAD_P12M', 'CUST_STAYS_CONRAD_P36M', 'CUST_STAYS_CURIO_P12M', 
                     'CUST_STAYS_CURIO_P36M', 'CUST_STAYS_WALDORF_P12M', 'CUST_STAYS_WALDORF_P36M', 
                     'CUST_STAYS_LXR_P12M', 'CUST_STAYS_LXR_P36M', 'CUST_STAYS_MOTTO_P12M', 
                     'CUST_STAYS_MOTTO_P36M', 'CUST_STAYS_SIGNIA_P12M', 'CUST_STAYS_SIGNIA_P36M', 
                     'CUST_STAYS_TEMPO_P12M', 'CUST_STAYS_TEMPO_P36M', 'CUST_FOLIO_HAMPTON_P12M', 
                     'CUST_FOLIO_HAMPTON_P36M', 'CUST_FOLIO_HILTON_P12M', 'CUST_FOLIO_HILTON_P36M', 
                     'CUST_FOLIO_HGI_P12M', 'CUST_FOLIO_HGI_P36M', 'CUST_FOLIO_HOMEWOOD_P12M', 
                     'CUST_FOLIO_HOMEWOOD_P36M', 'CUST_FOLIO_DOUBLETREE_P12M', 'CUST_FOLIO_DOUBLETREE_P36M', 
                     'CUST_FOLIO_EMBASSY_P12M', 'CUST_FOLIO_EMBASSY_P36M', 'CUST_FOLIO_HOME2_P12M', 
                     'CUST_FOLIO_HOME2_P36M', 'CUST_FOLIO_TRU_P12M', 'CUST_FOLIO_TRU_P36M', 
                     'CUST_FOLIO_CANOPY_P12M', 'CUST_FOLIO_CANOPY_P36M', 'CUST_FOLIO_TAPESTRY_P12M', 
                     'CUST_FOLIO_TAPESTRY_P36M', 'CUST_FOLIO_CONRAD_P12M', 'CUST_FOLIO_CONRAD_P36M', 
                     'CUST_FOLIO_CURIO_P12M', 'CUST_FOLIO_CURIO_P36M', 'CUST_FOLIO_WALDORF_P12M', 
                     'CUST_FOLIO_WALDORF_P36M', 'CUST_FOLIO_LXR_P12M', 'CUST_FOLIO_LXR_P36M', 
                     'CUST_FOLIO_MOTTO_P12M', 'CUST_FOLIO_MOTTO_P36M', 'CUST_FOLIO_SIGNIA_P12M', 
                     'CUST_FOLIO_SIGNIA_P36M', 'CUST_FOLIO_TEMPO_P12M', 'CUST_FOLIO_TEMPO_P36M', 
                     'CUST_NIGHTS_HAMPTON_P12M', 'CUST_NIGHTS_HAMPTON_P36M', 'CUST_NIGHTS_HILTON_P12M', 
                     'CUST_NIGHTS_HILTON_P36M', 'CUST_NIGHTS_HGI_P12M', 'CUST_NIGHTS_HGI_P36M', 
                     'CUST_NIGHTS_HOMEWOOD_P12M', 'CUST_NIGHTS_HOMEWOOD_P36M', 
                     'CUST_NIGHTS_DOUBLETREE_P12M', 
                     'CUST_NIGHTS_DOUBLETREE_P36M', 'CUST_NIGHTS_EMBASSY_P12M', 'CUST_NIGHTS_EMBASSY_P36M', 
                     'CUST_NIGHTS_HOME2_P12M', 'CUST_NIGHTS_HOME2_P36M', 'CUST_NIGHTS_TRU_P12M', 
                     'CUST_NIGHTS_TRU_P36M', 'CUST_NIGHTS_CANOPY_P12M', 'CUST_NIGHTS_CANOPY_P36M', 
                     'CUST_NIGHTS_TAPESTRY_P12M', 'CUST_NIGHTS_TAPESTRY_P36M', 'CUST_NIGHTS_CONRAD_P12M', 
                     'CUST_NIGHTS_CONRAD_P36M', 'CUST_NIGHTS_CURIO_P12M', 'CUST_NIGHTS_CURIO_P36M', 
                     'CUST_NIGHTS_WALDORF_P12M', 'CUST_NIGHTS_WALDORF_P36M', 'CUST_NIGHTS_LXR_P12M', 
                     'CUST_NIGHTS_LXR_P36M', 'CUST_NIGHTS_MOTTO_P12M', 'CUST_NIGHTS_MOTTO_P36M', 
                     'CUST_NIGHTS_SIGNIA_P12M', 'CUST_NIGHTS_SIGNIA_P36M', 'CUST_NIGHTS_TEMPO_P12M', 
                     'CUST_NIGHTS_TEMPO_P36M', 'COBRAND_CARD_SPEND_P12M', 'COBRAND_CARD_SPEND_36M', 
                     'COBRAND_CARD_HILTON_SPEND_P12M', 'COBRAND_CARD_HILTON_SPEND_P36M', 
                     'CUST_STAYS_DIRECT_P12M', 
                     'CUST_STAYS_DIRECT_P36M', 'CUST_STAYS_INDIRECT_P12M', 'CUST_STAYS_INDIRECT_P36M', 
                     'CUST_STAYS_OTA_P12M', 'CUST_STAYS_OTA_P36M', 'CUST_STAYS_OTHER_P12M', 
                     'CUST_STAYS_OTHER_P36M',
                     'CUST_MASS_PROMO_STAYS_P12M', 'CUST_MASS_PROMO_STAYS_P36M', 
                     'CUST_MASS_PROMO_NIGHTS_P12M', 
                     'CUST_MASS_PROMO_NIGHTS_P36M', 'TOTAL_DOMESTIC_TRIP_DIST_MILES_P36M', 
                     'TOTAL_INTL_TRIP_DIST_MILES_P36M']

In [0]:
# y = x[["HONORS_ACTIVE_FLAG", "DISTINCT_COUNTRIES_PAST_12MONTHS", "US_HONORS_INCOME_RANGE_DETAILED", 
#        "US_HONORS_HOUSEHOLD_SIZE", 'EMAIL_CLICKS_P36M', 'HONORS_TIER', 'MARKETABLE_CUSTOMER', 
#        'COBRAND_CARDHOLDER_FLAG', 'PRIMARY_STAY_TYPE_LIFETIME',  'Diff_FCY_NOW',
#        'PRIMARY_CHANNEL_LIFETIME', 'PROMO_REGISTRANT_FLAG_P12M', 'PREFERRED_LANGUGAGE_GROUPING',
#        'CUST_MASS_PROMO_STAYS_P36M', 'CUST_MASS_PROMO_NIGHTS_P36M', 'UNIQUE_BRANDS_P36M_BUCKETS',
#        'PT_REDEEMER_PAST_12MONTHS', 'CUST_STAYS_RESORT_P36M', 'CUST_STAYS_LUXURY_BRAND_DIV_P36M', 
#        'CUST_STAYS_P36M', 'FB_FOLIO_RATIO', 'FOLIO_NIGHTS_RATIO', 'TRIP_MILES_PER_STAY']]

# newdata2 = pd.concat([newdata.reset_index(drop=True), 
#                       pd.get_dummies(customers[['CUSTOMER_SEGMENT', 'COBRAND_SEG', 'LIFETIME_DIAMOND_FLAG', 'ORGANIC_INORGANIC_FLAG', 
#            'ENROLL_CHANNEL', 'APP_LAUNCHER_P12M', 'CUSTOMER_REGION_CODE', 'CUSTOMER_COUNTRY', 'FB_FOLIO_RATIO',
#            'HGV_OWNER_FLAG', 'PRIMARY_BRAND_LT', 'US_HONORS_OCCUPATION', 'US_HONORS_OCCUPATION_DETAILED', 
#            'US_HONORS_LIFE_STAGE', 'UPSCALE_PROPENSITY', 'POINTS_BALANCE_CATEGORIES',
#            'UNIQUE_BRANDS_P12M_BUCKETS', 'UNIQUE_BRANDS_P36M_BUCKETS', 'FIRST_PREFERRED_BOOKING_RATE_P36M', 
#            'SECOND_PREFERRED_BOOKING_RATE_P36M', 'THIRD_PREFERRED_BOOKING_RATE_P31M']])])

In [0]:
from pyspark.sql.functions import col, to_timestamp, date_format

df_with_string_dates = Value_df.withColumn("FIRST_CHECKOUT_YEAR", date_format(to_timestamp(col("FIRST_CHECKOUT_YEAR")), "yyyy-MM-dd HH:mm:ss")) \
    .withColumn("COBRAND_ENROLL_YEAR_POST_2018", date_format(to_timestamp(col("COBRAND_ENROLL_YEAR_POST_2018")), "yyyy-MM-dd HH:mm:ss")) \
    .withColumn("HONORS_ENROLL_YEAR", date_format(to_timestamp(col("HONORS_ENROLL_YEAR")), "yyyy-MM-dd HH:mm:ss"))


In [0]:
# Convert the Spark DataFrame to a Pandas DataFrame
customers = df_with_string_dates.select('HONORS_ACTIVE_FLAG', 'HONORS_TIER', 'DISTINCT_COUNTRIES_PAST_12MONTHS', 
                      'MARKETABLE_CUSTOMER', 'COBRAND_CARDHOLDER_FLAG', 'PRIMARY_STAY_TYPE_LIFETIME', 
                      'PRIMARY_CHANNEL_LIFETIME', 'HONORS_ENROLL_YEAR', 'FIRST_CHECKOUT_YEAR', 'COBRAND_ENROLL_YEAR_POST_2018',
                      'PROMO_REGISTRANT_FLAG_P12M', 'PREFERRED_LANGUGAGE_GROUPING', 
                      'US_HONORS_INCOME_RANGE_DETAILED', 'US_HONORS_HOUSEHOLD_SIZE', 
                      'PT_REDEEMER_PAST_12MONTHS',
                      'CUST_STAYS_P36M', 'CUST_NIGHTS_P36M', 'CUST_FOLIO_P36M', 'CUST_STAYS_RESORT_P36M', 
                      'CUST_STAYS_LUXURY_BRAND_DIV_P36M',  
                      'TOTAL_TRIP_DIST_MILES_P36M', 'EMAIL_CLICKS_P36M', 'CUSTOMER_SEGMENT', 'COBRAND_SEG', 'LIFETIME_DIAMOND_FLAG', 'ORGANIC_INORGANIC_FLAG', 
           'ENROLL_CHANNEL', 'APP_LAUNCHER_P12M', 'CUSTOMER_REGION_CODE', 'CUSTOMER_COUNTRY',
           'HGV_OWNER_FLAG', 'PRIMARY_BRAND_LT', 'US_HONORS_OCCUPATION', 'US_HONORS_OCCUPATION_DETAILED', 
           'US_HONORS_LIFE_STAGE', 'UPSCALE_PROPENSITY', 'POINTS_BALANCE_CATEGORIES',
           'UNIQUE_BRANDS_P12M_BUCKETS', 'UNIQUE_BRANDS_P36M_BUCKETS', 'FIRST_PREFERRED_BOOKING_RATE_P36M', 
           'SECOND_PREFERRED_BOOKING_RATE_P36M', 'THIRD_PREFERRED_BOOKING_RATE_P31M', 
 'CUST_AVG_PTS_REDEEMED_STAY_P12M',
 'CUSTOMER_VALUE_FY2022',
 'CUSTOMER_VALUE_3YEARS',
 'REWARD_HH_STAYS_P12M',
 'REWARD_HH_STAYS_P36M',
 'CUST_UNIQUE_BRANDS_P12M',
 'CUST_UNIQUE_BRANDS_P36M',
 'CUST_STAYS_P12M',
 'CUST_NIGHTS_P12M',
 'CUST_FOLIO_P12M',
 'CUST_FOOD_SPEND_P12M',
 'CUST_BEVERAGE_SPEND_P12M',
 'CUST_FOOD_SPEND_P36M',
 'CUST_BEVERAGE_SPEND_P36M',
 'CUST_STAYS_RESORT_P12M',
 'CUST_STAYS_LUXURY_BRAND_DIV_P12M',
 'CUST_DOMESTIC_STAYS_P12M',
 'CUST_INTL_STAYS_P12M',
 'CUST_DOMESTIC_STAYS_P36M',
 'CUST_INTL_STAYS_P36M',
 'TOTAL_TRIP_DIST_MILES_P12M',
 'TOTAL_DOMESTIC_TRIP_DIST_MILES_P12M',
 'TOTAL_INTL_TRIP_DIST_MILES_P12M',
 'EMAIL_CLICKS_P12M',
 'CUST_STAYS_HAMPTON_P12M',
 'CUST_STAYS_HAMPTON_P36M',
 'CUST_STAYS_HILTON_P12M',
 'CUST_STAYS_HILTON_P36M',
 'CUST_STAYS_HGI_P12M',
 'CUST_STAYS_HGI_P36M',
 'CUST_STAYS_HOMEWOOD_P12M',
 'CUST_STAYS_HOMEWOOD_P36M',
 'CUST_STAYS_DOUBLETREE_P12M',
 'CUST_STAYS_DOUBLETREE_P36M',
 'CUST_STAYS_EMBASSY_P12M',
 'CUST_STAYS_EMBASSY_P36M',
 'CUST_STAYS_HOME2_P12M',
 'CUST_STAYS_HOME2_P36M',
 'CUST_STAYS_TRU_P12M',
 'CUST_STAYS_TRU_P36M',
 'CUST_STAYS_CANOPY_P12M',
 'CUST_STAYS_CANOPY_P36M',
 'CUST_STAYS_TAPESTRY_P12M',
 'CUST_STAYS_TAPESTRY_P36M',
 'CUST_STAYS_CONRAD_P12M',
 'CUST_STAYS_CONRAD_P36M',
 'CUST_STAYS_CURIO_P12M',
 'CUST_STAYS_CURIO_P36M',
 'CUST_STAYS_WALDORF_P12M',
 'CUST_STAYS_WALDORF_P36M',
 'CUST_STAYS_LXR_P12M',
 'CUST_STAYS_LXR_P36M',
 'CUST_STAYS_MOTTO_P12M', 
 'CUST_STAYS_MOTTO_P36M',
 'CUST_STAYS_SIGNIA_P12M', 
 'CUST_STAYS_SIGNIA_P36M',
 'CUST_STAYS_TEMPO_P12M', 
 'CUST_STAYS_TEMPO_P36M', 
 'CUST_FOLIO_HAMPTON_P12M',
 'CUST_FOLIO_HAMPTON_P36M',
 'CUST_FOLIO_HILTON_P12M',
 'CUST_FOLIO_HILTON_P36M',
 'CUST_FOLIO_HGI_P12M',
 'CUST_FOLIO_HGI_P36M',
 'CUST_FOLIO_HOMEWOOD_P12M',
 'CUST_FOLIO_HOMEWOOD_P36M',
 'CUST_FOLIO_DOUBLETREE_P12M',
 'CUST_FOLIO_DOUBLETREE_P36M',
 'CUST_FOLIO_EMBASSY_P12M',
 'CUST_FOLIO_EMBASSY_P36M',
 'CUST_FOLIO_HOME2_P12M',
 'CUST_FOLIO_HOME2_P36M',
 'CUST_FOLIO_TRU_P12M',
 'CUST_FOLIO_TRU_P36M',
 'CUST_FOLIO_CANOPY_P12M',
 'CUST_FOLIO_CANOPY_P36M',
 'CUST_FOLIO_TAPESTRY_P12M',
 'CUST_FOLIO_TAPESTRY_P36M',
 'CUST_FOLIO_CONRAD_P12M',
 'CUST_FOLIO_CONRAD_P36M',
 'CUST_FOLIO_CURIO_P12M',
 'CUST_FOLIO_CURIO_P36M',
 'CUST_FOLIO_WALDORF_P12M',
 'CUST_FOLIO_WALDORF_P36M',
 'CUST_FOLIO_LXR_P12M',
 'CUST_FOLIO_LXR_P36M',
 'CUST_FOLIO_MOTTO_P12M',
 'CUST_FOLIO_MOTTO_P36M',
 'CUST_FOLIO_SIGNIA_P12M', 
 'CUST_FOLIO_SIGNIA_P36M',
 'CUST_FOLIO_TEMPO_P12M', 
 'CUST_FOLIO_TEMPO_P36M', 
 'CUST_NIGHTS_HAMPTON_P12M',
 'CUST_NIGHTS_HAMPTON_P36M',
 'CUST_NIGHTS_HILTON_P12M',
 'CUST_NIGHTS_HILTON_P36M',
 'CUST_NIGHTS_HGI_P12M',
 'CUST_NIGHTS_HGI_P36M',
 'CUST_NIGHTS_HOMEWOOD_P12M',
 'CUST_NIGHTS_HOMEWOOD_P36M',
 'CUST_NIGHTS_DOUBLETREE_P12M',
 'CUST_NIGHTS_DOUBLETREE_P36M',
 'CUST_NIGHTS_EMBASSY_P12M',
 'CUST_NIGHTS_EMBASSY_P36M',
 'CUST_NIGHTS_HOME2_P12M',
 'CUST_NIGHTS_HOME2_P36M',
 'CUST_NIGHTS_TRU_P12M',
 'CUST_NIGHTS_TRU_P36M',
 'CUST_NIGHTS_CANOPY_P12M',
 'CUST_NIGHTS_CANOPY_P36M',
 'CUST_NIGHTS_TAPESTRY_P12M',
 'CUST_NIGHTS_TAPESTRY_P36M',
 'CUST_NIGHTS_CONRAD_P12M',
 'CUST_NIGHTS_CONRAD_P36M',
 'CUST_NIGHTS_CURIO_P12M',
 'CUST_NIGHTS_CURIO_P36M',
 'CUST_NIGHTS_WALDORF_P12M',
 'CUST_NIGHTS_WALDORF_P36M',
 'CUST_NIGHTS_LXR_P12M',
 'CUST_NIGHTS_LXR_P36M',
 'CUST_NIGHTS_MOTTO_P12M', 
 'CUST_NIGHTS_MOTTO_P36M', 
 'CUST_NIGHTS_SIGNIA_P12M', 
 'CUST_NIGHTS_SIGNIA_P36M', 
 'CUST_NIGHTS_TEMPO_P12M', 
 'CUST_NIGHTS_TEMPO_P36M', 
 'COBRAND_CARD_SPEND_P12M',
 'COBRAND_CARD_SPEND_36M',
 'COBRAND_CARD_HILTON_SPEND_P12M',
 'COBRAND_CARD_HILTON_SPEND_P36M',
 'CUST_STAYS_DIRECT_P12M',
 'CUST_STAYS_DIRECT_P36M',
 'CUST_STAYS_INDIRECT_P12M',
 'CUST_STAYS_INDIRECT_P36M',
 'CUST_STAYS_OTA_P12M',
 'CUST_STAYS_OTA_P36M',
 'CUST_STAYS_OTHER_P12M',
 'CUST_STAYS_OTHER_P36M',
 'CUST_MASS_PROMO_STAYS_P12M',
 'CUST_MASS_PROMO_STAYS_P36M',
 'CUST_MASS_PROMO_NIGHTS_P12M',
 'CUST_MASS_PROMO_NIGHTS_P36M',
 'TOTAL_DOMESTIC_TRIP_DIST_MILES_P36M',
 'TOTAL_INTL_TRIP_DIST_MILES_P36M'
).toPandas()

In [0]:
# columns_to_fill = ['CUSTOMER_SEGMENT', 'COBRAND_SEG', 'LIFETIME_DIAMOND_FLAG', 'ORGANIC_INORGANIC_FLAG', 
#            'ENROLL_CHANNEL', 'APP_LAUNCHER_P12M', 'CUSTOMER_REGION_CODE', 'CUSTOMER_COUNTRY',
#            'HGV_OWNER_FLAG', 'PRIMARY_BRAND_LT', 'US_HONORS_OCCUPATION', 'US_HONORS_OCCUPATION_DETAILED', 
#            'US_HONORS_LIFE_STAGE', 'UPSCALE_PROPENSITY', 'POINTS_BALANCE_CATEGORIES',
#            'UNIQUE_BRANDS_P12M_BUCKETS', 'UNIQUE_BRANDS_P36M_BUCKETS', 'FIRST_PREFERRED_BOOKING_RATE_P36M', 
#            'SECOND_PREFERRED_BOOKING_RATE_P36M', 'THIRD_PREFERRED_BOOKING_RATE_P31M', 'PRIMARY_CHANNEL_LIFETIME']
# customers[columns_to_fill].fillna('UNKNOWN',inplace=True)

In [0]:
customers = customers.fillna({"ORGANIC_INORGANIC_FLAG": "UNKNOWN", 
              "CUSTOMER_SEGMENT": "UNKNOWN", 
              "LIFETIME_DIAMOND_FLAG": "UNKNOWN", 
              "APP_LAUNCHER_P12M": "UNKNOWN", 
              "HGV_OWNER_FLAG": "UNKNOWN", 
              "US_HONORS_OCCUPATION": "UNKNOWN", 
              "UNIQUE_BRANDS_P12M_BUCKETS": "UNKNOWN", 
              "UNIQUE_BRANDS_P36M_BUCKETS": "UNKNOWN", 
              "US_HONORS_OCCUPATION_DETAILED": "UNKNOWN",
              "US_HONORS_LIFE_STAGE": "UNKNOWN", 
              "POINTS_BALANCE_CATEGORIES": "UNKNOWN", 
              "FIRST_PREFERRED_BOOKING_RATE_P36M": "UNKNOWN",
              "SECOND_PREFERRED_BOOKING_RATE_P36M": "UNKNOWN",
              "THIRD_PREFERRED_BOOKING_RATE_P31M": "UNKNOWN",
              "ENROLL_CHANNEL": "UNKNOWN",
              "COBRAND_SEG": "UNKNOWN",
              "CUSTOMER_REGION_CODE": "UNKNOWN",
              "CUSTOMER_COUNTRY": "UNKNOWN",
              "PRIMARY_BRAND_LT": "UNKNOWN",
              "PRIMARY_CHANNEL_LIFETIME": "UNKNOWN",
              "UPSCALE_PROPENSITY": "UNKNOWN"
            })

In [0]:
# customers = customers.fillna({"FIRST_CHECKOUT_YEAR": '2100-01-01', 
#               "COBRAND_ENROLL_YEAR_POST_2018": '2100-01-01', 
#               "HONORS_ENROLL_YEAR": '2100-01-01'
#               })

In [0]:
columns_to_fill = [
 'CUST_STAYS_P36M',
 'CUST_NIGHTS_P36M',
 'CUST_FOLIO_P36M',
 'CUST_STAYS_RESORT_P36M',
 'CUST_STAYS_LUXURY_BRAND_DIV_P36M',
 'TOTAL_TRIP_DIST_MILES_P36M',
 'EMAIL_CLICKS_P36M',
 'CUST_AVG_PTS_REDEEMED_STAY_P12M',
 'CUSTOMER_VALUE_FY2022',
 'CUSTOMER_VALUE_3YEARS',
 'REWARD_HH_STAYS_P12M',
 'REWARD_HH_STAYS_P36M',
 'CUST_UNIQUE_BRANDS_P12M',
 'CUST_UNIQUE_BRANDS_P36M',
 'CUST_STAYS_P12M',
 'CUST_NIGHTS_P12M',
 'CUST_FOLIO_P12M',
 'CUST_FOOD_SPEND_P12M',
 'CUST_BEVERAGE_SPEND_P12M',
 'CUST_FOOD_SPEND_P36M',
 'CUST_BEVERAGE_SPEND_P36M',
 'CUST_STAYS_RESORT_P12M',
 'CUST_STAYS_LUXURY_BRAND_DIV_P12M',
 'CUST_DOMESTIC_STAYS_P12M',
 'CUST_INTL_STAYS_P12M',
 'CUST_DOMESTIC_STAYS_P36M',
 'CUST_INTL_STAYS_P36M',
 'TOTAL_TRIP_DIST_MILES_P12M',
 'TOTAL_DOMESTIC_TRIP_DIST_MILES_P12M',
 'TOTAL_INTL_TRIP_DIST_MILES_P12M',
 'EMAIL_CLICKS_P12M',
 'CUST_STAYS_HAMPTON_P12M',
 'CUST_STAYS_HAMPTON_P36M',
 'CUST_STAYS_HILTON_P12M',
 'CUST_STAYS_HILTON_P36M',
 'CUST_STAYS_HGI_P12M',
 'CUST_STAYS_HGI_P36M',
 'CUST_STAYS_HOMEWOOD_P12M',
 'CUST_STAYS_HOMEWOOD_P36M',
 'CUST_STAYS_DOUBLETREE_P12M',
 'CUST_STAYS_DOUBLETREE_P36M',
 'CUST_STAYS_EMBASSY_P12M',
 'CUST_STAYS_EMBASSY_P36M',
 'CUST_STAYS_HOME2_P12M',
 'CUST_STAYS_HOME2_P36M',
 'CUST_STAYS_TRU_P12M',
 'CUST_STAYS_TRU_P36M',
 'CUST_STAYS_CANOPY_P12M',
 'CUST_STAYS_CANOPY_P36M',
 'CUST_STAYS_TAPESTRY_P12M',
 'CUST_STAYS_TAPESTRY_P36M',
 'CUST_STAYS_CONRAD_P12M',
 'CUST_STAYS_CONRAD_P36M',
 'CUST_STAYS_CURIO_P12M',
 'CUST_STAYS_CURIO_P36M',
 'CUST_STAYS_WALDORF_P12M',
 'CUST_STAYS_WALDORF_P36M',
 'CUST_STAYS_LXR_P12M',
 'CUST_STAYS_LXR_P36M',
 'CUST_STAYS_MOTTO_P12M', 
 'CUST_STAYS_MOTTO_P36M',
 'CUST_STAYS_SIGNIA_P12M', 
 'CUST_STAYS_SIGNIA_P36M',
 'CUST_STAYS_TEMPO_P12M', 
 'CUST_STAYS_TEMPO_P36M', 
 'CUST_FOLIO_HAMPTON_P12M',
 'CUST_FOLIO_HAMPTON_P36M',
 'CUST_FOLIO_HILTON_P12M',
 'CUST_FOLIO_HILTON_P36M',
 'CUST_FOLIO_HGI_P12M',
 'CUST_FOLIO_HGI_P36M',
 'CUST_FOLIO_HOMEWOOD_P12M',
 'CUST_FOLIO_HOMEWOOD_P36M',
 'CUST_FOLIO_DOUBLETREE_P12M',
 'CUST_FOLIO_DOUBLETREE_P36M',
 'CUST_FOLIO_EMBASSY_P12M',
 'CUST_FOLIO_EMBASSY_P36M',
 'CUST_FOLIO_HOME2_P12M',
 'CUST_FOLIO_HOME2_P36M',
 'CUST_FOLIO_TRU_P12M',
 'CUST_FOLIO_TRU_P36M',
 'CUST_FOLIO_CANOPY_P12M',
 'CUST_FOLIO_CANOPY_P36M',
 'CUST_FOLIO_TAPESTRY_P12M',
 'CUST_FOLIO_TAPESTRY_P36M',
 'CUST_FOLIO_CONRAD_P12M',
 'CUST_FOLIO_CONRAD_P36M',
 'CUST_FOLIO_CURIO_P12M',
 'CUST_FOLIO_CURIO_P36M',
 'CUST_FOLIO_WALDORF_P12M',
 'CUST_FOLIO_WALDORF_P36M',
 'CUST_FOLIO_LXR_P12M',
 'CUST_FOLIO_LXR_P36M',
 'CUST_FOLIO_MOTTO_P12M',
 'CUST_FOLIO_MOTTO_P36M',
 'CUST_FOLIO_SIGNIA_P12M', 
 'CUST_FOLIO_SIGNIA_P36M',
 'CUST_FOLIO_TEMPO_P12M', 
 'CUST_FOLIO_TEMPO_P36M', 
 'CUST_NIGHTS_HAMPTON_P12M',
 'CUST_NIGHTS_HAMPTON_P36M',
 'CUST_NIGHTS_HILTON_P12M',
 'CUST_NIGHTS_HILTON_P36M',
 'CUST_NIGHTS_HGI_P12M',
 'CUST_NIGHTS_HGI_P36M',
 'CUST_NIGHTS_HOMEWOOD_P12M',
 'CUST_NIGHTS_HOMEWOOD_P36M',
 'CUST_NIGHTS_DOUBLETREE_P12M',
 'CUST_NIGHTS_DOUBLETREE_P36M',
 'CUST_NIGHTS_EMBASSY_P12M',
 'CUST_NIGHTS_EMBASSY_P36M',
 'CUST_NIGHTS_HOME2_P12M',
 'CUST_NIGHTS_HOME2_P36M',
 'CUST_NIGHTS_TRU_P12M',
 'CUST_NIGHTS_TRU_P36M',
 'CUST_NIGHTS_CANOPY_P12M',
 'CUST_NIGHTS_CANOPY_P36M',
 'CUST_NIGHTS_TAPESTRY_P12M',
 'CUST_NIGHTS_TAPESTRY_P36M',
 'CUST_NIGHTS_CONRAD_P12M',
 'CUST_NIGHTS_CONRAD_P36M',
 'CUST_NIGHTS_CURIO_P12M',
 'CUST_NIGHTS_CURIO_P36M',
 'CUST_NIGHTS_WALDORF_P12M',
 'CUST_NIGHTS_WALDORF_P36M',
 'CUST_NIGHTS_LXR_P12M',
 'CUST_NIGHTS_LXR_P36M',
 'CUST_NIGHTS_MOTTO_P12M', 
 'CUST_NIGHTS_MOTTO_P36M', 
 'CUST_NIGHTS_SIGNIA_P12M', 
 'CUST_NIGHTS_SIGNIA_P36M', 
 'CUST_NIGHTS_TEMPO_P12M', 
 'CUST_NIGHTS_TEMPO_P36M', 
 'COBRAND_CARD_SPEND_P12M',
 'COBRAND_CARD_SPEND_36M',
 'COBRAND_CARD_HILTON_SPEND_P12M',
 'COBRAND_CARD_HILTON_SPEND_P36M',
 'CUST_STAYS_DIRECT_P12M',
 'CUST_STAYS_DIRECT_P36M',
 'CUST_STAYS_INDIRECT_P12M',
 'CUST_STAYS_INDIRECT_P36M',
 'CUST_STAYS_OTA_P12M',
 'CUST_STAYS_OTA_P36M',
 'CUST_STAYS_OTHER_P12M',
 'CUST_STAYS_OTHER_P36M',
 'CUST_MASS_PROMO_STAYS_P12M',
 'CUST_MASS_PROMO_STAYS_P36M',
 'CUST_MASS_PROMO_NIGHTS_P12M',
 'CUST_MASS_PROMO_NIGHTS_P36M',
 'TOTAL_DOMESTIC_TRIP_DIST_MILES_P36M',
 'TOTAL_INTL_TRIP_DIST_MILES_P36M']

# customers[columns_to_fill] = customers[columns_to_fill].apply(pd.to_numeric, errors='coerce')
# customers[columns_to_fill].fillna(0, inplace=True)

customers[columns_to_fill] = customers[columns_to_fill].apply(pd.to_numeric, errors='coerce')
customers[columns_to_fill] = customers[columns_to_fill].fillna(0)

In [0]:
customers['US_HONORS_LIFE_STAGE'] = customers['US_HONORS_LIFE_STAGE'].replace(['NA', '5. NA', 'None'], 'UNKNOWN')
customers['US_HONORS_OCCUPATION'] = customers['US_HONORS_OCCUPATION'].replace(['NA', '13. NA', '99'], 'UNKNOWN')
customers['US_HONORS_OCCUPATION_DETAILED'] = customers['US_HONORS_OCCUPATION_DETAILED'].replace(['NA', '13. NA'], 'UNKNOWN')
customers['ENROLL_CHANNEL'] = customers['ENROLL_CHANNEL'].replace(['N/A'], 'UNKNOWN')
customers['CUSTOMER_REGION_CODE'] = customers['CUSTOMER_REGION_CODE'].replace(['NA'], 'UNKNOWN')
customers['CUSTOMER_COUNTRY'] = customers['CUSTOMER_COUNTRY'].replace(['NA'], 'UNKNOWN')
customers['PRIMARY_CHANNEL_LIFETIME'] = customers['PRIMARY_CHANNEL_LIFETIME'].replace(['N/A'], 'UNKNOWN BRAND')
customers['UPSCALE_PROPENSITY'] = customers['UPSCALE_PROPENSITY'].replace(['NA', '4. NA'], 'UNKNOWN')
customers['PT_REDEEMER_PAST_12MONTHS'] = customers['PT_REDEEMER_PAST_12MONTHS'].replace(['NA', '4. NA'], 'UNKNOWN')

In [0]:
customers = customers.fillna({"FIRST_CHECKOUT_YEAR": '2100-01-01', 
              "COBRAND_ENROLL_YEAR_POST_2018": '2100-01-01', 
              "HONORS_ENROLL_YEAR": '2100-01-01'
              })


In [0]:
# # Find all columns with null values in the DataFram
# null_columns = customers.columns[customers.isnull().any()].tolist()
# null_columns

In [0]:
len(clustering_columns), len(profiling_columns)

(25, 164)

In [0]:
customers['FB_FOLIO_RATIO'] = (customers['CUST_FOOD_SPEND_P36M'] + customers['CUST_BEVERAGE_SPEND_P36M']) / customers['CUST_FOLIO_P36M']
customers['FB_FOLIO_RATIO'].fillna(0,inplace=True)

/home/spark-b9ed5074-0197-4f36-9eed-a1/.ipykernel/2610/command-2854201248256283-1116374802:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.




In [0]:
x = customers[clustering_columns]

In [0]:
x["US_HONORS_INCOME_RANGE_DETAILED"].fillna(70000,inplace=True)
x["US_HONORS_HOUSEHOLD_SIZE"].fillna(99,inplace=True)
x["CUST_STAYS_P36M"].fillna(0,inplace=True)
x["CUST_NIGHTS_P36M"].fillna(0,inplace=True)
x["CUST_FOLIO_P36M"].fillna(0,inplace=True)
x["EMAIL_CLICKS_P36M"].fillna(0,inplace=True)
x["CUST_STAYS_RESORT_P36M"].fillna(0,inplace=True)
x["CUST_STAYS_LUXURY_BRAND_DIV_P36M"].fillna(0,inplace=True)

/home/spark-b9ed5074-0197-4f36-9eed-a1/.ipykernel/2610/command-2854201248256285-1346184628:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


/home/spark-b9ed5074-0197-4f36-9eed-a1/.ipykernel/2610/command-2854201248256285-1346184628:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For 

In [0]:
x['FIRST_CHECKOUT_YEAR'] = x['FIRST_CHECKOUT_YEAR']\
    .mask(x['FIRST_CHECKOUT_YEAR'] == '9999-12-31 00:00:00', '2100-01-01')\
    .str.split(' ')\
    .str[0]\
    .astype('datetime64[ns]')
x['COBRAND_ENROLL_YEAR_POST_2018'] = x['COBRAND_ENROLL_YEAR_POST_2018']\
    .mask(x['COBRAND_ENROLL_YEAR_POST_2018'] == '0999-12-31 00:00:00', '2100-01-01')\
    .str.split(' ')\
    .str[0]\
    .astype('datetime64[ns]')
x['HONORS_ENROLL_YEAR'] = x['HONORS_ENROLL_YEAR']\
    .mask(x['HONORS_ENROLL_YEAR'] == '9999-12-31 00:00:00', '2100-01-01')\
    .str.split(' ')\
    .str[0]\
    .astype('datetime64[ns]')

In [0]:
# x.columns[x.isnull().any()].tolist()

In [0]:
x["PT_REDEEMER_PAST_12MONTHS"] = list(map(lambda v: v 
                              if v in ('NON PT REDEEMER', 'PT REDEEMER')
                              else 'Unknown', x["PT_REDEEMER_PAST_12MONTHS"]))
x["PREFERRED_LANGUGAGE_GROUPING"] = list(map(lambda v: v 
                              if v in ("English", "Not English")
                              else 'Unknown', x["PREFERRED_LANGUGAGE_GROUPING"]))
x["PROMO_REGISTRANT_FLAG_P12M"] = list(map(lambda v: v 
                              if v in ('NOT PROMO REGISTRANT', 'PROMO REGISTRANT')
                              else 'Unknown', x["PROMO_REGISTRANT_FLAG_P12M"]))
# calculate time difference between the datedey to 1/1/2024
x['Diff_FCY_NOW'] = ((pd.to_datetime("2024-01-01") - x['FIRST_CHECKOUT_YEAR'])/365).dt.days
x.loc[x['Diff_FCY_NOW']<0, 'Diff_FCY_NOW'] = 0
# Cap CUST_STAYS_P36M to 50
x.loc[x['CUST_STAYS_P36M']>50, 'CUST_STAYS_P36M'] = 50
x.loc[x['CUST_STAYS_RESORT_P36M']>20, 'CUST_STAYS_RESORT_P36M'] = 20
x.loc[x['CUST_STAYS_LUXURY_BRAND_DIV_P36M']>20, 'CUST_STAYS_LUXURY_BRAND_DIV_P36M'] = 20
x['FOLIO_NIGHTS_RATIO'] = x['CUST_FOLIO_P36M'] / x['CUST_NIGHTS_P36M']
x['FOLIO_NIGHTS_RATIO'].fillna(0,inplace=True)

x['TRIP_MILES_PER_STAY'] = x['TOTAL_TRIP_DIST_MILES_P36M'] / x['CUST_STAYS_P36M']
x['TRIP_MILES_PER_STAY'].fillna(0,inplace=True)
x["HONORS_ACTIVE_FLAG"] = list(map(lambda v: "Inactive Honors" 
                              if v == "Inactive Honors - Closed Accounts" 
                              else v, x["HONORS_ACTIVE_FLAG"]))
x["PRIMARY_STAY_TYPE_LIFETIME"] = list(map(lambda v: v 
                              if v in ("Other/Mix", "Business", "Leisure")
                              else "Unknown", x["PRIMARY_STAY_TYPE_LIFETIME"]))
x["PRIMARY_CHANNEL_LIFETIME"] = list(map(lambda v: v 
                              if v in ("Direct", "OTA", "Other/Mix", "GDS")
                              else "Unknown", x["PRIMARY_CHANNEL_LIFETIME"]))
DISTINCT_COUNTRIES_lookup = {
    "Zero countries": 0,
    "1 country": 1,
    "2 countries": 2,
    "3 or more countries": 3
}
x["DISTINCT_COUNTRIES_PAST_12MONTHS"] = x["DISTINCT_COUNTRIES_PAST_12MONTHS"].map(DISTINCT_COUNTRIES_lookup) # re-mapping
#x["DISTINCT_COUNTRIES_PAST_12MONTHS"] = list(map(lambda v: "2+ countries" 
#                              if v in ("2 countries", "3 or more countries")
#                              else v, x["DISTINCT_COUNTRIES_PAST_12MONTHS"]))
x["US_HONORS_INCOME_RANGE_DETAILED"] = list(map(lambda v: "70000" 
                              if v in ("English", "Not English")
                              else v, x["US_HONORS_INCOME_RANGE_DETAILED"]))
x["US_HONORS_INCOME_RANGE_DETAILED"] = x["US_HONORS_INCOME_RANGE_DETAILED"].astype(int)
x["US_HONORS_HOUSEHOLD_SIZE"] = list(map(lambda v: "99" 
                              if v not in ("1. <2", "2. 2-4", "3. >4")
                              else v, x["US_HONORS_HOUSEHOLD_SIZE"]))
HH_Size_lookup = {
    "1. <2": 1,
    "2. 2-4": 3,
    "3. >4": 5,
    "99": 2
}
x["US_HONORS_HOUSEHOLD_SIZE"] = x["US_HONORS_HOUSEHOLD_SIZE"].map(HH_Size_lookup) # re-mapping

/home/spark-b9ed5074-0197-4f36-9eed-a1/.ipykernel/2610/command-2854201248256288-47972508:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


/home/spark-b9ed5074-0197-4f36-9eed-a1/.ipykernel/2610/command-2854201248256288-47972508:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For ex

In [0]:
x['FOLIO_STAYS_RATIO'] = x['CUST_FOLIO_P36M'] / x['CUST_STAYS_P36M']
x['FOLIO_STAYS_RATIO'].fillna(0,inplace=True)

# x['FB_FOLIO_RATIO'] = (x['CUST_FOOD_SPEND_P36M'] + x['CUST_BEVERAGE_SPEND_P36M']) / x['CUST_FOLIO_P36M']
# x['FB_FOLIO_RATIO'].fillna(0,inplace=True)

/home/spark-b9ed5074-0197-4f36-9eed-a1/.ipykernel/2610/command-2854201248256289-1663896509:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.




In [0]:
# Original Features

y = x[["HONORS_ACTIVE_FLAG", "DISTINCT_COUNTRIES_PAST_12MONTHS", "US_HONORS_INCOME_RANGE_DETAILED", 
       "US_HONORS_HOUSEHOLD_SIZE", 'EMAIL_CLICKS_P36M', 'HONORS_TIER', 'MARKETABLE_CUSTOMER', 
       'COBRAND_CARDHOLDER_FLAG', 'PRIMARY_STAY_TYPE_LIFETIME', 'Diff_FCY_NOW',
       'PRIMARY_CHANNEL_LIFETIME', 'PROMO_REGISTRANT_FLAG_P12M', 'PREFERRED_LANGUGAGE_GROUPING',
       'PT_REDEEMER_PAST_12MONTHS', 'CUST_STAYS_RESORT_P36M', 'CUST_STAYS_LUXURY_BRAND_DIV_P36M', 
        'CUST_STAYS_P36M', 'FOLIO_NIGHTS_RATIO', 'TRIP_MILES_PER_STAY',
        'FB_FOLIO_RATIO']]

In [0]:
# y = x[["HONORS_ACTIVE_FLAG", "DISTINCT_COUNTRIES_PAST_12MONTHS", "US_HONORS_INCOME_RANGE_DETAILED", 
#        "US_HONORS_HOUSEHOLD_SIZE", 'EMAIL_CLICKS_P36M', 'HONORS_TIER', 'MARKETABLE_CUSTOMER', 
#        'COBRAND_CARDHOLDER_FLAG', 'PRIMARY_STAY_TYPE_LIFETIME',  'Diff_FCY_NOW',
#        'PRIMARY_CHANNEL_LIFETIME', 'PROMO_REGISTRANT_FLAG_P12M', 'PREFERRED_LANGUGAGE_GROUPING',
#        'CUST_MASS_PROMO_STAYS_P36M', 'CUST_MASS_PROMO_NIGHTS_P36M', 'UNIQUE_BRANDS_P12M_BUCKETS', 'UNIQUE_BRANDS_P36M_BUCKETS',
#        'PT_REDEEMER_PAST_12MONTHS', 'CUST_STAYS_RESORT_P36M', 'CUST_STAYS_LUXURY_BRAND_DIV_P36M', 
#        'CUST_STAYS_P36M', 'FB_FOLIO_RATIO', 'FOLIO_NIGHTS_RATIO', 'TRIP_MILES_PER_STAY']]

In [0]:
x[["FIRST_CHECKOUT_YEAR", "COBRAND_ENROLL_YEAR_POST_2018", "HONORS_ENROLL_YEAR"]] = x[["FIRST_CHECKOUT_YEAR", "COBRAND_ENROLL_YEAR_POST_2018", "HONORS_ENROLL_YEAR"]].astype(str)

In [0]:
newdata=pd.get_dummies(y)

In [0]:
newdata

,DISTINCT_COUNTRIES_PAST_12MONTHS,US_HONORS_INCOME_RANGE_DETAILED,US_HONORS_HOUSEHOLD_SIZE,EMAIL_CLICKS_P36M,MARKETABLE_CUSTOMER,Diff_FCY_NOW,CUST_STAYS_RESORT_P36M,CUST_STAYS_LUXURY_BRAND_DIV_P36M,CUST_STAYS_P36M,FOLIO_NIGHTS_RATIO,TRIP_MILES_PER_STAY,FB_FOLIO_RATIO,HONORS_ACTIVE_FLAG_Active Honors,HONORS_ACTIVE_FLAG_Inactive Honors,HONORS_ACTIVE_FLAG_Non Honors,HONORS_TIER_B,HONORS_TIER_D,HONORS_TIER_G,HONORS_TIER_NHH,HONORS_TIER_S,COBRAND_CARDHOLDER_FLAG_COBRAND CARDHOLDER,COBRAND_CARDHOLDER_FLAG_NOT COBRAND CARDHOLDER,PRIMARY_STAY_TYPE_LIFETIME_Business,PRIMARY_STAY_TYPE_LIFETIME_Leisure,PRIMARY_STAY_TYPE_LIFETIME_Other/Mix,PRIMARY_STAY_TYPE_LIFETIME_Unknown,PRIMARY_CHANNEL_LIFETIME_Direct,PRIMARY_CHANNEL_LIFETIME_GDS,PRIMARY_CHANNEL_LIFETIME_OTA,PRIMARY_CHANNEL_LIFETIME_Other/Mix,PRIMARY_CHANNEL_LIFETIME_Unknown,PROMO_REGISTRANT_FLAG_P12M_NOT PROMO REGISTRANT,PROMO_REGISTRANT_FLAG_P12M_PROMO REGISTRANT,PROMO_REGISTRANT_FLAG_P12M_Unknown,PREFERRED_LANGUGAGE_GROUPING_English,PREFERRED_LANGUGAGE_GROUPING_Not English,PREFERRED_LANGUGAGE_GROUPING_Unknown,PT_REDEEMER_PAST_12MONTHS_NON PT REDEEMER,PT_REDEEMER_PAST_12MONTHS_PT REDEEMER,PT_REDEEMER_PAST_12MONTHS_Unknown
0,1,125000,3,17.0,1,6,2,0,10,132.200556,914.538071,0.116876,True,False,False,False,False,False,False,True,False,True,True,False,False,False,True,False,False,False,False,False,True,False,True,False,False,False,True,False
1,0,0,1,0.0,1,6,0,0,1,380.756667,210.825144,0.056904,True,False,False,True,False,False,False,False,False,True,False,False,True,False,False,False,False,True,False,True,False,False,True,False,False,True,False,False
2,1,0,1,2.0,0,6,0,0,4,122.075000,0.000000,0.000000,False,True,False,False,False,False,True,False,False,True,True,False,False,False,True,False,False,False,False,True,False,False,True,False,False,True,False,False
3,1,87500,1,5.0,1,6,0,0,5,108.053158,451.299193,0.000000,False,True,False,False,False,False,False,True,False,True,False,True,False,False,True,False,False,False,False,True,False,False,True,False,False,True,False,False
4,2,70000,2,19.0,0,6,0,0,12,190.398333,0.000000,0.083540,True,False,False,False,False,True,False,False,False,True,True,False,False,False,True,False,False,False,False,False,True,False,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6264074,1,70000,2,0.0,0,1,0,0,1,180.420000,0.000000,0.000000,False,False,True,False,False,False,True,False,False,True,True,False,False,False,False,False,True,False,False,True,False,False,False,True,False,True,False,False
6264075,1,70000,2,0.0,0,1,0,0,1,134.000000,0.000000,0.000000,False,False,True,False,False,False,True,False,False,True,True,False,False,False,True,False,False,False,False,True,False,False,False,True,False,True,False,False
6264076,1,70000,2,0.0,0,1,0,0,1,82.780000,0.000000,0.000000,False,False,True,False,False,False,True,False,False,True,False,False,True,False,True,False,False,False,False,True,False,False,False,True,False,True,False,False
6264077,1,70000,2,0.0,0,1,0,0,1,155.000000,0.000000,0.000000,False,False,True,False,False,False,True,False,False,True,False,False,True,False,True,False,False,False,False,True,False,False,False,True,False,True,False,False


In [0]:
# For unstandardized values

def standardize_column(df, col):
    mean = df[col].mean()
    std = df[col].std()
    df[col] = (df[col] - mean) / std
    return mean, std

In [0]:
standardize=["DISTINCT_COUNTRIES_PAST_12MONTHS", "US_HONORS_INCOME_RANGE_DETAILED",
       "US_HONORS_HOUSEHOLD_SIZE", 'EMAIL_CLICKS_P36M', 'CUST_STAYS_RESORT_P36M', 'Diff_FCY_NOW', 
       'CUST_STAYS_LUXURY_BRAND_DIV_P36M', 'CUST_STAYS_P36M', 'FOLIO_NIGHTS_RATIO', 'TRIP_MILES_PER_STAY',
       'FB_FOLIO_RATIO']

# standardize=["DISTINCT_COUNTRIES_PAST_12MONTHS", "US_HONORS_INCOME_RANGE_DETAILED",
#        "US_HONORS_HOUSEHOLD_SIZE", 'EMAIL_CLICKS_P36M', 'CUST_STAYS_RESORT_P36M', 'Diff_FCY_NOW', 
#        'CUST_STAYS_LUXURY_BRAND_DIV_P36M', 'CUST_STAYS_P36M', 'FOLIO_NIGHTS_RATIO', 'TRIP_MILES_PER_STAY',
#        'FB_FOLIO_RATIO']

mean_std_dict = {}
for col in standardize:
    newdata[col] = y[col]  # Ensure newdata has the correct columns from y
    mean, std = standardize_column(newdata, col)
    mean_std_dict[col] = (mean, std)

In [0]:
cols=newdata.columns.tolist()
cols

['DISTINCT_COUNTRIES_PAST_12MONTHS',
 'US_HONORS_INCOME_RANGE_DETAILED',
 'US_HONORS_HOUSEHOLD_SIZE',
 'EMAIL_CLICKS_P36M',
 'MARKETABLE_CUSTOMER',
 'Diff_FCY_NOW',
 'CUST_STAYS_RESORT_P36M',
 'CUST_STAYS_LUXURY_BRAND_DIV_P36M',
 'CUST_STAYS_P36M',
 'FOLIO_NIGHTS_RATIO',
 'TRIP_MILES_PER_STAY',
 'FB_FOLIO_RATIO',
 'HONORS_ACTIVE_FLAG_Active Honors',
 'HONORS_ACTIVE_FLAG_Inactive Honors',
 'HONORS_ACTIVE_FLAG_Non Honors',
 'HONORS_TIER_B',
 'HONORS_TIER_D',
 'HONORS_TIER_G',
 'HONORS_TIER_NHH',
 'HONORS_TIER_S',
 'COBRAND_CARDHOLDER_FLAG_COBRAND CARDHOLDER',
 'COBRAND_CARDHOLDER_FLAG_NOT COBRAND CARDHOLDER',
 'PRIMARY_STAY_TYPE_LIFETIME_Business',
 'PRIMARY_STAY_TYPE_LIFETIME_Leisure',
 'PRIMARY_STAY_TYPE_LIFETIME_Other/Mix',
 'PRIMARY_STAY_TYPE_LIFETIME_Unknown',
 'PRIMARY_CHANNEL_LIFETIME_Direct',
 'PRIMARY_CHANNEL_LIFETIME_GDS',
 'PRIMARY_CHANNEL_LIFETIME_OTA',
 'PRIMARY_CHANNEL_LIFETIME_Other/Mix',
 'PRIMARY_CHANNEL_LIFETIME_Unknown',
 'PROMO_REGISTRANT_FLAG_P12M_NOT PROMO REGISTRAN

In [0]:
newdata2 = pd.concat([newdata.reset_index(drop=True), 
                      pd.get_dummies(customers[['CUSTOMER_SEGMENT', 'COBRAND_SEG', 'LIFETIME_DIAMOND_FLAG', 'ORGANIC_INORGANIC_FLAG', 
           'ENROLL_CHANNEL', 'APP_LAUNCHER_P12M', 'CUSTOMER_REGION_CODE', 'CUSTOMER_COUNTRY', 
           'HGV_OWNER_FLAG', 'PRIMARY_BRAND_LT', 'US_HONORS_OCCUPATION', 'US_HONORS_OCCUPATION_DETAILED', 
           'US_HONORS_LIFE_STAGE', 'UPSCALE_PROPENSITY', 'POINTS_BALANCE_CATEGORIES',
           'UNIQUE_BRANDS_P12M_BUCKETS', 'UNIQUE_BRANDS_P36M_BUCKETS', 'FIRST_PREFERRED_BOOKING_RATE_P36M', 
           'SECOND_PREFERRED_BOOKING_RATE_P36M', 'THIRD_PREFERRED_BOOKING_RATE_P31M']]),
           customers[columns_to_fill]], axis=1)

In [0]:
# newdata2 = pd.concat([newdata.reset_index(drop=True), 
#                       pd.get_dummies(customers[['CUSTOMER_SEGMENT', 'COBRAND_SEG', 'LIFETIME_DIAMOND_FLAG', 'ORGANIC_INORGANIC_FLAG', 
#            'ENROLL_CHANNEL', 'APP_LAUNCHER_P12M', 'CUSTOMER_REGION_CODE', 'CUSTOMER_COUNTRY', 
#            'HGV_OWNER_FLAG', 'PRIMARY_BRAND_LT', 'US_HONORS_OCCUPATION', 'US_HONORS_OCCUPATION_DETAILED', 
#            'US_HONORS_LIFE_STAGE', 'UPSCALE_PROPENSITY', 'POINTS_BALANCE_CATEGORIES',
#            'FIRST_PREFERRED_BOOKING_RATE_P36M', 
#            'SECOND_PREFERRED_BOOKING_RATE_P36M', 'THIRD_PREFERRED_BOOKING_RATE_P31M']]),
#            customers[columns_to_fill]], axis=1)

In [0]:
X=newdata[cols].to_numpy()
TSS = np.sum((X-X.mean(0))**2)
stats=[]

for k in range(8,9):

    #Clustering
    clusters=KMeans(n_clusters=k,random_state=1234,n_init=50,max_iter=100000).fit(newdata[cols])
    ss=silhouette_score(newdata[cols],clusters.labels_,sample_size=10000)
    stats.append({'k':k,'silhouette':ss,'within/total':clusters.inertia_/TSS})

    centroids = clusters.cluster_centers_

    newdata2['clusters']=clusters.labels_
    cluster_counts = newdata2.groupby('clusters').size()
    cluster_percentages = cluster_counts / len(newdata)

    #Profiling
    profiles=newdata2.groupby('clusters').mean().transpose().reset_index()
    profiles.columns=['variable']+[f'class{i+1}'for i in range(k)]

    profiles = profiles.drop_duplicates(subset='variable')

    # Unstandardize the profiling variables
    for col in standardize:
        col_mean, col_std = mean_std_dict[col]
        profiles.loc[profiles['variable'] == col, [f'class{i+1}' for i in range(k)]] = \
            profiles.loc[profiles['variable'] == col, [f'class{i+1}' for i in range(k)]].apply(lambda x: x * col_std + col_mean)

    # Add row with cluster percentages
    profiles.loc[len(profiles)] = ['Proportion'] + cluster_percentages.tolist()
    
    profiles.to_csv(f'Value_segmentation_profiling_variables_{k}_{dt.date.today()}.csv',index=False)

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f5e04440360>
Traceback (most recent call last):
  File "/databricks/python/lib/python3.11/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/databricks/python/lib/python3.11/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/python/lib/python3.11/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
                   ^^^^^^^^^^^^^^^^^^
  File "/databricks/python/lib/python3.11/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
             ^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'split'
Ex

In [0]:
newdata2['clusters'].unique()

array([5, 4, 3, 2, 1, 0], dtype=int32)

In [0]:
stats=pd.DataFrame(stats)
stats=pd.melt(stats,id_vars=['k'],value_vars=['silhouette','within/total'])
p1=(ggplot(stats,aes('k','value',color='factor(variable)'))
+ geom_point()
+ geom_line()
+ xlab("Number of Clusters"))

In [0]:
p1.save(filename='clustering_results.png')

/local_disk0/.ephemeral_nfs/envs/pythonEnv-a1846ca2-8408-47e2-9daa-42fa58355e20/lib/python3.11/site-packages/plotnine/ggplot.py:606: PlotnineWarning: Saving 6.4 x 4.8 in image.
/local_disk0/.ephemeral_nfs/envs/pythonEnv-a1846ca2-8408-47e2-9daa-42fa58355e20/lib/python3.11/site-packages/plotnine/ggplot.py:607: PlotnineWarning: Filename: clustering_results.png
/local_disk0/.ephemeral_nfs/envs/pythonEnv-a1846ca2-8408-47e2-9daa-42fa58355e20/lib/python3.11/site-packages/plotnine/geoms/geom_path.py:113: PlotnineWarning: geom_path: Each group consist of only one observation. Do you need to adjust the group aesthetic?
